# Exploratory Data Analysis

In [ ]:
import os
import time
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import make_grid
from torchvision import transforms
from collections import defaultdict
from torchvision.datasets.folder import pil_loader
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pylab
from skimage import io, transform

pd.set_option('max_colwidth', 800)

%matplotlib inline

## Load Data

We have seven categories of musculoskeletal radiographs

In [ ]:
train_df = pd.read_csv('../MURA-v1.0/train.csv', names=['Path', 'Label'])
valid_df = pd.read_csv('../MURA-v1.0/valid.csv', names=['Path', 'Label'])

Let's checkout the shapes of dataframes

In [ ]:
train_df.shape, valid_df.shape

We have 37111 radiographs for training and 3225 radiographs for validation set, let's peak into the dataframes

In [ ]:
train_df.head(3)

In [ ]:
valid_df.head(3)

So, we have radiograph paths and their correspoinding labels, each radiographs has a label of 0 (normal) or 1 (abnormal)

## Analysis

According to paper: <br>
1.

    The MURA abnormality detection task is a binary classification task, where the input is an upper 
    exremity radiograph study — with each study containing one or more views (images) — and the 
    expected output is a binary label y ∈ {0, 1} indicating whether the "study" is normal or abnormal, 
    respectively.
2.

    The model takes as input one or more views for a study of an upper extremity. On each view, our 169-
    layer convolutional neural network predicts the probability of abnormality. We compute the overall 
    probability of abnormality for the study by taking the arithmetic mean of the abnormality 
    probabilities output by the network for each image. The model makes the binary prediction of 
    abnormal if the probability of abnormality for the study is greater than 0.5.

So, we have make predictions on study level, taking into account the predictions of all the views (images) of the study. This can be done by taking arithmetic mean of all the views (images) under a particular study.

In [ ]:
train_df.head(30)

Analyzing this dataframe, we can see that images are annotated based on whether their corresponding study is positive (normal, 0) or negative (abnormal, 1)

### Plot some random radiographs from training and validation set

In [ ]:
train_mat = train_df.as_matrix()
valid_mat = valid_df.as_matrix()

In [ ]:
ix = np.random.randint(0, len(train_mat)) # randomly select a index
img_path = train_mat[ix][0]
plt.imshow(io.imread(img_path), cmap='binary')
cat = img_path.split('/')[2] # get the radiograph category
plt.title('Category: %s & Lable: %d ' %(cat, train_mat[ix][1]))
plt.show()

In [ ]:
ix = np.random.randint(0, len(valid_mat))
img_path = valid_mat[ix][0]
plt.imshow(io.imread(img_path), cmap='binary')
cat = img_path.split('/')[2]
plt.title('Category: %s & Lable: %d ' %(cat, valid_mat[ix][1]))
plt.show()

This can be seen that images vary in resolution and dimension

In [ ]:
# look at the pixel values
io.imread(img_path)[0]

### Data Exploration

In [ ]:
!ls ../MURA-v1.0/train/

In [ ]:
!ls ../MURA-v1.0/train/XR_ELBOW/

So, train dataset has seven study types, each study type has studies on patients stored in folders named like patient001, patient002 etc..

#### Patient count per study type

Let's count number of patients in each study type

In [ ]:
data_cat= ['train', 'valid']
study_types = list(os.walk('../MURA-v1.0/train/'))[0][1] # study types, same for train and valid sets
patients_count = {}  # to store all patients count for each study type, for train and valid sets
for phase in data_cat:
    patients_count[phase] = {}
    for study_type in study_types:
        patients = list(os.walk('../MURA-v1.0/%s/%s' %(phase, study_type)))[0][1] # patient folder names
        patients_count[phase][study_type] = len(patients)

In [ ]:
print(study_types)
print()
print(patients_count)

In [ ]:
# plot the patient counts per study type 

fig, ax = plt.subplots(figsize=(10, 5))
for i, phase in enumerate(data_cat):
    counts = patients_count[phase].values()
    m = max(counts)
    for i, v in enumerate(counts):
        if v==m: ax.text(i-0.1, v+3, str(v))
        else: ax.text(i-0.1, v + 20, str(v))
    x_pos = np.arange(len(study_types))
    plt.bar(x_pos, counts, alpha=0.5)
    plt.xticks(x_pos, study_types)

plt.xlabel('Study types')
plt.ylabel('Number of patients')
plt.legend(['train', 'valid'])
plt.show()
fig.savefig('images/pcpst.jpg', bbox_inches='tight', pad_inches=0) # name=patient count per study type

XR_FINGER has got the most number of patients (1867 in train set, 166 in valid set) followed by XR_WRIST

### Study count

Patients might have multiple studies for a given study type, like a patient may have two studies for wrist, independent of each other. <br> Let's have a look at such cases, **NOTE** here study count = number of patients which have same number of studies

In [ ]:
# let's find out number of studies per study_type
study_count = {} # to store study counts for each study type 
for study_type in study_types:
    BASE_DIR = '../MURA-v1.0/train/%s/' % study_type
    study_count[study_type] = defaultdict(lambda:0) # to store study count for current study_type, initialized to 0 by default
    patients = list(os.walk(BASE_DIR))[0][1] # patient folder names
    for patient in patients:
        studies = os.listdir(BASE_DIR+patient)
        study_count[study_type][len(studies)] += 1

In [ ]:
study_count

XR_WRIST has 3111 patients who have only single study, similarly, 158 patients have 2 studies, 12 patients have 3 studies and 4 patients have 4 studies. <br> let's plot this data

In [ ]:
# plot the study count vs number of patients per study type data 
fig = plt.figure(figsize=(8, 25))
for i, study_type in enumerate(study_count):
    ax = fig.add_subplot(7, 1, i+1)
    study = study_count[study_type]
    # text in the plot
    m = max(study.values())
    for i, v in enumerate(study.values()):
        if v==m: ax.text(i-0.1, v - 200, str(v))
        else: ax.text(i-0.1, v + 200, str(v))
    ax.text(i, m - 100, study_type, color='green')
    # plot the bar chart
    x_pos = np.arange(len(study))
    plt.bar(x_pos, study.values(), align='center', alpha=0.5)
    plt.xticks(x_pos,  study.keys())
    plt.xlabel('Study count')
    plt.ylabel('Number of patients')
plt.show()
fig.savefig('images/pcpsc.jpg', bbox_inches='tight', pad_inches=0)

### Number of views per study

It can be seen that each study may have more that one view (radiograph image), let' have a look

In [ ]:
# let's find out number of studies per study_type
view_count = {} # to store study counts for each study type, study count = number of patients which have similar number of studies 
for study_type in study_types:
    BASE_DIR = '../MURA-v1.0/train/%s/' % study_type
    view_count[study_type] = defaultdict(lambda:0) # to store study count for current study_type, initialized to 0 by default
    patients = list(os.walk(BASE_DIR))[0][1] # patient folder names
    for patient in patients:
        studies = os.listdir(BASE_DIR + patient)
        for study in studies:
            views = os.listdir(BASE_DIR + patient + '/' + study)
            view_count[study_type][len(views)] += 1

In [ ]:
view_count

`XR_SHOULDER` has as many as 13 views in some studies, `XR_HAND` has 5 at max, this poses a challenging task to predict on a study taking into account all the views of that study while keeping the batch size of 8 (as mentioned in MURA paper)

In [ ]:
# plot the view count vs number of studies per study type data 
fig = plt.figure(figsize=(10, 30))
for i, view_type in enumerate(view_count):
    ax = fig.add_subplot(7, 1, i+1)
    view = view_count[view_type]
    # text in the plot
    m = max(view.values())
    for i, v in enumerate(view.values()):
        if v==m: ax.text(i-0.1, v - 200, str(v))
        else: ax.text(i-0.1, v + 80, str(v))
    ax.text(i - 0.5, m - 80, view_type, color='green')
    # plot the bar chart
    x_pos = np.arange(len(view))
    plt.bar(x_pos, view.values(), align='center', alpha=0.5)
    plt.xticks(x_pos,  view.keys())
    plt.xlabel('Number of views')
    plt.ylabel('Number of studies')
plt.show()
fig.savefig('images/nsvc.jpg', bbox_inches='tight', pad_inches=0) # name=number of studies view count

Most of the studies contain 2, 3 or 4 views